<a href="https://colab.research.google.com/github/RogerHeederer/NLP_entry/blob/master/SentencePiece.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference Source : wikidocs.net 유영준님 자료

스스로 학습하면서 필요한 부분에는 추가적 설명, 소스 코드 삽입 및 수정 등이 있습니다. 영리적 목적이 아닌, 자기 계발 목적으로 정리한 자료입니다.

In [1]:
##센텐스피스##

**센텐스피스는 사전 토큰화 작업없이 단어 분리 토큰화를 수행한다. 언어에 종속되지 않음**

In [2]:
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 2.8MB/s 


In [3]:
import sentencepiece as spm
import pandas as pd
import urllib.request
import csv

In [4]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x7f67788bb128>)

In [6]:
train_df = pd.read_csv('IMDb_Reviews.csv')
train_df['review']

0        My family and I normally do not watch local mo...
1        Believe it or not, this was at one time the wo...
2        After some internet surfing, I found the "Home...
3        One of the most unheralded great works of anim...
4        It was the Sixties, and anyone with long hair ...
                               ...                        
49995    the people who came up with this are SICK AND ...
49996    The script is so so laughable... this in turn,...
49997    "So there's this bride, you see, and she gets ...
49998    Your mind will not be satisfied by this nobud...
49999    The chaser's war on everything is a weekly sho...
Name: review, Length: 50000, dtype: object

In [7]:
print('리뷰 개수 :', len(train_df))

리뷰 개수 : 50000


In [8]:
#센텐스피스의 입력으로 사용하기 위해서 데이터 프레임을 txt로 저장한다
with open('imdb_review.txt', 'w', encoding='utf8') as f:
  f.write('\n'.join(train_df['review']))

In [9]:
#센텐스피스로 단어 집합과 각 단어에 고유한 정수 부여
spm.SentencePieceTrainer.Train('--input=imdb_review.txt --model_prefix=imdb --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

각 인자가 의미하는 바
* input : 학습시킬 파일
* model_prefix : 만들어질 모델 이름
* vocab_size : 단어 집합의 크기
* model_type : 사용할 모델 (unigram(default), bpe, char, word)
* max_sentence_length: 문장의 최대 길이
* pad_id, pad_piece: pad token id, 값
* unk_id, unk_piece: unknown token id, 값
* bos_id, bos_piece: begin of sentence token id, 값
* eos_id, eos_piece: end of sequence token id, 값
* user_defined_symbols: 사용자 정의 토큰


In [10]:
vocab_list = pd.read_csv('imdb.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list.sample(10)

,0,1
3576,▁system,-3573
3755,dered,-3752
229,ard,-226
4219,olds,-4216
2064,head,-2061
4238,zen,-4235
224,▁has,-221
940,▁worst,-937
1598,chn,-1595
722,▁around,-719


In [11]:
vocab_list.shape

(5000, 2)

In [12]:
sp = spm.SentencePieceProcessor()
vocab_file = "imdb.model"
sp.load(vocab_file)

True

In [13]:
# encode_as_pieces : 문장을 입력하면 서브 워드 시퀀스로 변환
# encode_as_ids : 문장을 입력하면 정수 시퀀스로 변환

lines = [
         "I didn't at all think of it this way.",
         "I have waited a long time for someone to film"
]
for line in lines:
  print(line)
  print(sp.encode_as_pieces(line))
  print(sp.encode_as_ids(line))
  print()

I didn't at all think of it this way.
['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.']
[41, 623, 4950, 4926, 138, 169, 378, 30, 58, 73, 413, 4945]

I have waited a long time for someone to film
['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91]



In [14]:
sp.GetPieceSize() # 단어집합 크기 확인

5000

In [16]:
sp.IdToPiece(430) #정수로부터 맵팽되는 서브워드 변환

'▁character'

In [20]:
sp.PieceToId('▁character') #서브워드로부터 맵핑되는 정수 변환

430

In [21]:
sp.DecodeIds([41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91]) #정수 시퀀스로부터 문장 변화

'I have waited a long time for someone to film'

In [22]:
#서브워드 시퀀스로부터 문장으로 변환
sp.DecodePieces(['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film'])

'I have waited a long time for someone to film'

In [23]:
print(sp.encode('I have waited a long time for someone to film', out_type=str))
print(sp.encode('I have waited a long time for someone to film', out_type=int))

['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91]


##네이버 영화 리뷰 토큰화하기##

In [24]:
import pandas as pd
import sentencepiece as spm
import urllib.request
import csv

In [25]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x7f676e3c8828>)

In [26]:
naver_df = pd.read_table('ratings.txt')
naver_df[:5]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [27]:
print('리뷰 개수 :', len(naver_df))

리뷰 개수 : 200000


In [28]:
print(naver_df.isnull().values.any())

True


In [29]:
naver_df = naver_df.dropna(how='any') # Null 값이 존재하는 행 제거
print(naver_df.isnull().values.any())

False


In [30]:
print('리뷰 개수 :', len(naver_df))

리뷰 개수 : 199992


In [31]:
with open('naver_review.txt', 'w', encoding='utf8') as f:
  f.write('\n'.join(naver_df['document']))

In [32]:
spm.SentencePieceTrainer.Train('--input=naver_review.txt --model_prefix=naver --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

In [33]:
vocab_list = pd.read_csv('naver.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list[:10]
#unknown, start, /start  스페셜 토큰 0,1,2 에 사용

,0,1
0,<unk>,0
1,<s>,0
2,</s>,0
3,..,0
4,영화,-1
5,▁영화,-2
6,▁이,-3
7,▁아,-4
8,...,-5
9,▁그,-6


In [34]:
vocab_list.sample(10)

,0,1
2445,▁영화지만,-2442
1873,▁병맛,-1870
3066,피소,-3063
4110,윌,-4107
2430,지컬,-2427
1249,▁19,-1246
3949,료,-3946
665,어도,-662
3709,딱,-3706
1766,▁훈훈,-1763


In [35]:
len(vocab_list)

5000

In [37]:
sp = spm.SentencePieceProcessor()
vocab_file = "naver.model"
sp.load(vocab_file)

True

In [38]:
lines = [
  "뭐 이딴 것도 영화냐.",
  "진짜 최고의 영화입니다 ㅋㅋ",
]
for line in lines:
  print(line)
  print(sp.encode_as_pieces(line))
  print(sp.encode_as_ids(line))
  print()

뭐 이딴 것도 영화냐.
['▁뭐', '▁이딴', '▁것도', '▁영화냐', '.']
[132, 966, 1296, 2590, 3276]

진짜 최고의 영화입니다 ㅋㅋ
['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 200, 821, 85]



In [39]:
sp.IdToPiece(4)

'영화'

In [40]:
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=str))
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=int))

['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 200, 821, 85]
